In [1]:
# the basic workflow for each document is as follows
# https://www.tensorflow.org/tutorials/load_data/text#train_the_model
# Author: Jackson Hajer

In [10]:
import os
from tensorflow.data import AUTOTUNE
from tensorflow import expand_dims
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense,GlobalMaxPooling1D,Embedding, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
fileloc_train = "./C50train/"
fileloc_test = "./C50test/"
articles_train = []
articles_test = []
labels_train = []
label_names = []

data_set_train = text_dataset_from_directory(fileloc_train,batch_size=50)
data_set_test = text_dataset_from_directory(fileloc_test,batch_size=50)

Found 2500 files belonging to 50 classes.
Found 2500 files belonging to 50 classes.


In [12]:
MAX_SEQUENCE_LENGTH = 250
VOCAB_SIZE = 10000
int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [18]:
train_text = data_set_train.map(lambda text, labels: text)
int_vectorize_layer.adapt(train_text)

In [19]:
def int_vectorize_text(text, label):
  text = expand_dims(text, -1)
  return int_vectorize_layer(text), label

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [20]:
int_train_ds = data_set_train.map(int_vectorize_text)
int_test_ds = data_set_test.map(int_vectorize_text)

In [21]:
int_train_ds = configure_dataset(int_train_ds)
int_test_ds = configure_dataset(int_test_ds)

In [35]:
def create_model(vocab_size, num_labels):
  model = Sequential([
      Embedding(vocab_size, 64, mask_zero=True),
      Conv1D(64, 5, padding="valid", activation="sigmoid", strides=2),
      GlobalMaxPooling1D(),
      Dense(50,activation='softmax')
  ])
  return model

In [38]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss= SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds,
                        epochs = 50, 
                        batch_size = 26, 
                        validation_data = int_test_ds)

Epoch 1/50
50/50 [==============================] - 2s 24ms/step - loss: 3.9751 - accuracy: 0.0172 - val_loss: 3.9171 - val_accuracy: 0.0200
Epoch 2/50
50/50 [==============================] - 1s 21ms/step - loss: 3.9046 - accuracy: 0.0272 - val_loss: 3.9051 - val_accuracy: 0.0224
Epoch 3/50
50/50 [==============================] - 1s 21ms/step - loss: 3.8815 - accuracy: 0.0860 - val_loss: 3.8914 - val_accuracy: 0.0536
Epoch 4/50
50/50 [==============================] - 1s 23ms/step - loss: 3.8417 - accuracy: 0.1892 - val_loss: 3.8618 - val_accuracy: 0.0800
Epoch 5/50
50/50 [==============================] - 1s 23ms/step - loss: 3.7774 - accuracy: 0.3192 - val_loss: 3.8172 - val_accuracy: 0.1304
Epoch 6/50
50/50 [==============================] - 1s 23ms/step - loss: 3.6925 - accuracy: 0.4608 - val_loss: 3.7659 - val_accuracy: 0.1700
Epoch 7/50
50/50 [==============================] - 1s 22ms/step - loss: 3.5925 - accuracy: 0.6108 - val_loss: 3.7088 - val_accuracy: 0.2192
Epoch 8/50
50